In [3]:
# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep
import sched, time
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *



In [4]:
# Pulling Yahoo live data

'''
Function for pulling latest SPX, VIX, VVIX, or SKEW data. Input is a string, pulls 
the latest 2 lines of data from yahoo finance for given ticker and returns a 
dataframe of the open and close with the latest date as the first row.
'''
def latest_yahoo(ticker = 'SPX'):
    # Using requests to ping yahoo finance to retrieve 
    # historical data table
    if ticker == 'VIX':
        site = 'https://finance.yahoo.com/quote/%5EVIX/history?p=^VIX'
    elif ticker == 'VVIX':
        site = 'https://finance.yahoo.com/quote/%5EVVIX/history?p=^VVIX'
    elif ticker == 'SKEW':
        site = 'https://finance.yahoo.com/quote/%5ESKEW/history?p=^SKEW'
    else:
        site = 'https://finance.yahoo.com/quote/%5EGSPC/history?p=^GSPC'
        
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    # Initializing list to store date, open, and close values
    # for GSPC
    dates = []
    opens = []
    closes = []
    
    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    end_row = 3
    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        
        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = [item.split('>')[1] for item in [string.split('</span>')[0] for string in individual_row[0].split('<span ')[1:]]]
        
        if i == 1:
            # Skip first row because they are column headers
            i += 1
            continue
        elif i == end_row:
            break
        else:
            # Append necessary items to initialized lists for 
            # dataframe storage
            dates.append(row_items[0])
            opens.append(float(row_items[1].replace(',','')))
            closes.append(float(row_items[5].replace(',','')))
        i += 1
    
    # Return dataframe of necessary values
    return pd.DataFrame({ticker + ' Open': opens,ticker + ' Close': closes}, index = dates)
    
'''
Function for pulling options data from yahoo Input is a string, either 'call' or 'put' to
determine the contract type to pull from yahoo finance. The output is a dataframe of the latest
data from yahoo finance tagged with the current date-time. Output columns are pull date-time,
contract name, strike, last price, bid, ask volume, open interest, and IV (in decimal form).
'''
def yahoo_options(contract = 'put'):
    # Using request to ping yahoo and retrieve the raw html
    # tables for calls and puts data for gspc
    site = 'https://finance.yahoo.com/quote/%5EGSPC/options?p=%5EGSPC'
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    calls = soup.find_all('table')[0]
    puts = soup.find_all('table')[1]
    if contract == 'call':
        table = calls
    else:
        table = puts
        
    # parsing calls table
    
    # initiating data lists for storing column data
    dates = []
    names = []
    strikes = []
    lprices = []
    bids = []
    asks = []
    volumes = []
    open_interests = []
    iv = []
    current_time = dt.datetime.now()
    
    # starting with counter i so that the first row in the
    # specified table is noted as the header row
    i = 1
    for row in table.find_all('tr'):
        rowstring = str(row).split('\n')
        # if first row, store data as header labels for dataframe use
        if i == 1:
            header = [data.split('>')[1] for data in \
                        [rawstring.split('</span>')[0] for \
                         rawstring in rowstring[0].split('<span')[1:]]]
        else:
        # parsing other raw row strings to retrieve necessary data and append them
        # to their corresponding list.
            row_data = rowstring[0].split('data-reactid=')[3:]
            dates.append(current_time)
            names.append(row_data[0].split('title="')[1].split('"')[0])
            strikes.append(float(row_data[3].split('</a>')[0].split('>')[1].replace(',','')))
            lprices.append(float(row_data[4].split('</td>')[0].split('>')[1].replace(',','')))
            bids.append(float(row_data[5].split('</td>')[0].split('>')[1].replace(',','')))
            asks.append(float(row_data[6].split('</td>')[0].split('>')[1].replace(',','')))
            volumes.append(float(row_data[-3].split('</td>')[0].split('>')[1].replace(',','')))
            open_interests.append(float(row_data[-2].split('</td>')[0].split('>')[1].replace(',','')))
            iv.append(float(row_data[-1].split('</td>')[0].split('>')[1].replace(',','').replace('%',''))/100)
        i += 1

    return pd.DataFrame({'Pull Date': dates,
                          contract + ' ' + header[0]: names,
                          header[2]: strikes,
                          header[3]: lprices,
                          header[4]: bids,
                          header[5]: asks,
                          header[-3]: volumes,
                          header[-2]: open_interests,
                          header[-1]: iv})

def get_options_data(ticker = '^GSPC'):
    tape = Options(ticker, 'yahoo')
    data = tape.get_all_data()
    data.to_csv('raw.csv')
    data = pd.read_csv('raw.csv')
    del data['JSON']
    return data

In [10]:
root = 'C:\\Users\\Fang\\Desktop\\Python Trading\\SPX Option Backtester\\SPX Put Selling\\Live Data Pulling\\Options Data\\'
i = 0
starttime = time.time()
mainfile = 'optionsdata.csv'

while True:
    df = get_options_data()
    if os.path.isfile(root + mainfile) == True:
        df.to_csv(root + mainfile, mode = 'a', header = False)
    else:
        df.to_csv(root + mainfile)
    df.to_csv(root + 'data' + str(i) + '.csv')    
    time.sleep(60 - ((time.time() - starttime) % 60))

KeyboardInterrupt: 